# Text Summarization Using OCR and Attention Networks
## Final Year Project (2018-2019)
### Guide: Mrs. Sneha H R
### Mentor: Dr. Prasanta Gogoi
### Members: Dhwani D Shah,Mitali Sandip Sheth,Nitisha Yadav, Pooja Pandit
                

## Importing libraries and classes

In [1]:
import os

import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from collections import Counter

import Summarizer
import summarizer_data_utils
import summarizer_model_utils

In [2]:
print(tf.__version__)

1.13.1


## The Data
The data we will use here is the 'all-the-news'-dataset from Kaggle. It contains about 200000 news articles and the headlines of those articles.The articles are from several big news corporations.

In [3]:
data = pd.read_csv('/home/mitali/toi.csv',
                   encoding='utf-8')
data1 = pd.read_csv('/home/mitali/articles2.csv',
                    encoding='utf-8')
data2 = pd.read_csv('/home/mitali/articles3.csv',
                    encoding='utf-8')
data3 = pd.read_csv('/home/mitali/articles1.csv',
                    encoding='utf-8')

In [4]:
data = pd.concat([data, data1, data2,data3])
data.shape

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


(143846, 10)

In [5]:
data.head()

,Unnamed: 0,author,content,date,id,month,publication,title,url,year
0,NaN,NaN,\nNEW DELHI: In a move to aid the real estate ...,NaN,NaN,NaN,NaN,\nGST Council meet: GST council approves trans...,NaN,NaN
1,NaN,NaN,"\nNEW DELHI: Infosys Ltd, India's second-bigge...",NaN,NaN,NaN,NaN,\nABN Amro: Infosys to buy 75% stake in ABN AM...,NaN,NaN
2,NaN,NaN,\nNEW DELHI: Lyft Inc on Wednesday raised the ...,NaN,NaN,NaN,NaN,\nLyft raises IPO price target as investor fre...,NaN,NaN
3,NaN,NaN,"\nNEW DELHI: On Friday, diamantaire Nirav Modi...",NaN,NaN,NaN,NaN,\n‘A golden visa can save you from going to ja...,NaN,NaN
4,NaN,NaN,\nNEW DELHI: \n\n Jet Airways\n + has started...,NaN,NaN,NaN,NaN,"\nJet Airways News: Planes grounded, Jet Airwa...",NaN,NaN


In [6]:
Counter(data.publication)

Counter({'Atlantic': 7179,
         'Breitbart': 23781,
         'Business Insider': 6757,
         'Buzzfeed News': 4854,
         'CNN': 11488,
         'Fox News': 4354,
         'Guardian': 8681,
         'NPR': 11992,
         'National Review': 6203,
         'New York Post': 17493,
         'New York Times': 7803,
         'Reuters': 10710,
         'Talking Points Memo': 5214,
         'Vox': 4947,
         'Washington Post': 11114,
         nan: 1276})

In [7]:
data = data[data.publication != 'Breitbart']

In [8]:
data.isnull().sum()

Unnamed: 0      1276
author         17152
content          119
date            3917
id              1276
month           3917
publication     1276
title              2
url            34506
year            3917
dtype: int64

In [9]:
len(data.title)

120065

In [10]:
data.title[0]

0    \nGST Council meet: GST council approves trans...
0    Patriots Day Is Best When It Digs Past the Her...
0    Alton Sterling’s son: ’Everyone needs to prote...
0    House Republicans Fret About Winning Their Hea...
Name: title, dtype: object

In [11]:
data.dropna(subset=['title'], inplace = True)

In [14]:
data.rename(index = str, columns = {'title':'Headline', 'content':'Story'}, inplace = True)
data = data[['Headline', 'Story']]

In [15]:
for x in data.Headline[:10]:
    print(x)


GST Council meet: GST council approves transition plan for new tax rates for housing sector - Times of India



ABN Amro: Infosys to buy 75% stake in ABN AMRO's mortgage services arm - Times of India



Lyft raises IPO price target as investor fret over missing out - Times of India



‘A golden visa can save you from going to jail at home’ - Times of India



Jet Airways News: Planes grounded, Jet Airways sends expat pilots on leave without pay 



Asian central banks look to cut rates after Fed pause - Times of India



Heineken buys 74L shares of UBHL from ED - Times of India



Tata Group: Tatas, GIC & SSG to buy stake in GMR Airports by investing Rs 8,000 crore 



Uber Careem Merger: Uber to buy Mideast rival Careem for $3.1 billion - Times of India



Talegaon - The Green Heaven for your Dream Home - Times of India




In [18]:
len_headline = [len(headline) for i, headline in enumerate(data.Headline)]


In [19]:
len_story=[]
for i,story in enumerate(data.Story):
    if isinstance(story,str):
        l=len(story)
        len_story.append((i,l))

In [20]:
len(len_story),len(len_headline)

(119944, 120063)

In [21]:
index = set(i for i in range(len(len_story)))
indexh= set(i for i in range(len(len_headline)))
indexd=indexh-index
len(indexh),len(index),len(indexd)

(120063, 119944, 119)

In [22]:
len_headline[:10], len_story[:10]

([111, 90, 82, 75, 90, 73, 60, 89, 89, 67],
 [(0, 2663),
  (1, 783),
  (2, 2826),
  (3, 5730),
  (4, 3105),
  (5, 1898),
  (6, 1266),
  (7, 3640),
  (8, 1426),
  (9, 4080)])

In [23]:
len_headline= [val for ind, val in enumerate(len_headline) if ind not in indexd]
len_story= [val for ind, val in enumerate(len_story) if ind not in indexd]

In [24]:
len(len_headline),len(len_story)

(119944, 119944)

In [25]:
len_headline[:10], len_story[:10]

([111, 90, 82, 75, 90, 73, 60, 89, 89, 67],
 [(0, 2663),
  (1, 783),
  (2, 2826),
  (3, 5730),
  (4, 3105),
  (5, 1898),
  (6, 1266),
  (7, 3640),
  (8, 1426),
  (9, 4080)])

In [26]:
len_headline_counted = Counter(len_headline).most_common()
len_story_counted = Counter(len_story).most_common()
len_data= len(data.Headline)
len_headline_counted[:10], len_story_counted[:10]

([(64, 2894),
  (63, 2893),
  (60, 2890),
  (59, 2864),
  (62, 2847),
  (58, 2838),
  (61, 2789),
  (65, 2784),
  (66, 2673),
  (67, 2651)],
 [((0, 2663), 1),
  ((1, 783), 1),
  ((2, 2826), 1),
  ((3, 5730), 1),
  ((4, 3105), 1),
  ((5, 1898), 1),
  ((6, 1266), 1),
  ((7, 3640), 1),
  ((8, 1426), 1),
  ((9, 4080), 1)])

## Training Dataset
Taking the articles whose content length is 500.

In [27]:
indices = [ind for ind, val in len_story if 50 < val < 1000]

In [28]:
indices[:10],len(indices)

([1, 11, 21, 45, 47, 48, 70, 75, 76, 77], 7736)

In [29]:
l1=set(i for i in range(len_data))
l2=set(indices)
u_id=list(l1-l2)

In [30]:
texts_unprocessed= [text for ind, text in enumerate(data.Story) if ind not in u_id]
summaries_unprocessed= [text for ind, text in enumerate(data.Headline) if ind not in u_id]

In [31]:
len(texts_unprocessed),len(u_id),len(summaries_unprocessed)

(7736, 112327, 7736)

In [32]:
to_remove = ['- The New York Times', '- Breitbart','- The Times Of India']

summaries_unprocessed_clean = []
texts_unprocessed_clean = []

removed = 0
append = True
for sentence in summaries_unprocessed:
    append = True
    for r in to_remove:
        if sentence.endswith(r):
            sentence = sentence.replace(r, '.')
            summaries_unprocessed_clean.append(sentence.replace(r, '.'))
            removed+=1
            append = False
            break
            
    if append:
        summaries_unprocessed_clean.append(sentence)

In [33]:
len(summaries_unprocessed_clean), len(texts_unprocessed)

(7736, 7736)

In [34]:
import nltk

## Clean and prepare data

In [35]:
processed_texts, processed_summaries, words_counted = summarizer_data_utils.preprocess_texts_and_summaries(
    texts_unprocessed,
    summaries_unprocessed_clean,
    keep_most=False)

Processing Time:  11.464321374893188


In [36]:
processed_texts_clean = []
processed_summaries_clean = []

for t, s in zip(processed_texts, processed_summaries):
    if t != [] and s != []:
        processed_texts_clean.append(t)
        processed_summaries_clean.append(s)

## Create lookup dictionaries
We cannot feed our network actual words, but numbers. So we first have to create our lookup dictionaries, where each word gets an int value (high or low, depending on its frequency in our corpus). Those help us to later convert the texts into numbers.

We also add special tokens. EndOfSentence(EOS) and StartOfSentence(SOS) are crucial for the Seq2Seq model we later use. Pad token(PAD), because all summaries and texts in a batch need to have the same length, pad token helps us do that.

Unknown Token(UNK) refers to certain words that might be proper nouns and are not frequently used.




In [37]:
specials = ["<EOS>", "<SOS>","<PAD>","<UNK>"]
word2ind, ind2word,  missing_words = summarizer_data_utils.create_word_inds_dicts(words_counted,
                                                                                  specials = specials,
                                                                                  min_occurences = 2)
print(len(word2ind), len(ind2word), len(missing_words))

23168 23168 14685


## Pretrained Embeddings
We can use pretrained word embeddings as they have proved to increase training speed and accuracy. Here I have used the embeddings from tf_hub. 

In [38]:

embed = hub.Module("https://tfhub.dev/google/Wiki-words-250/1")
emb = embed([key for key in word2ind.keys()])

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    embedding = sess.run(emb)

Instructions for updating:
Colocations handled automatically by placer.


W0402 15:18:29.574964 140079934850880 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0402 15:18:29.782527 140079934850880 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [39]:
embedding.shape

(23168, 250)

In [40]:
np.save('./tf_hub_embedding_headlinesA.npy', embedding)

## Convert Text and Summaries
As we cannot feed the words directly to our network, we have to convert them to numbers first. So we use the function to convert them to integer values. And we also append the SOS and EOS tokens.

In [41]:
# converts words in texts and summaries to indices
converted_texts, unknown_words_in_texts = summarizer_data_utils.convert_to_inds(processed_texts_clean,
                                                                                word2ind,
                                                                                eos = False)

In [42]:
converted_summaries, unknown_words_in_summaries = summarizer_data_utils.convert_to_inds(processed_summaries_clean,
                                                                                        word2ind,
                                                                                        eos = True,
                                                                                        sos = True)

In [43]:
print(processed_texts_clean[0])

['new', 'delhi', 'infosys', 'ltd', 'india', 's', 'second', 'biggest', 'it', 'services', 'company', 'said', 'on', 'thursday', 'it', 'would', 'buy', 'a', '75', 'per', 'cent', 'stake', 'in', 'abn', 'amro', 'group', 'nv', 's', 'mortgage', 'administration', 'services', 'unit', 'for', '127', '5', 'million', 'euros', '143', '53', 'million', 'infosys', 'will', 'acquire', 'the', 'stake', 'in', 'stater', 'nv', 'through', 'unit', 'infosys', 'consulting', 'pvt', 'ltd', 'and', 'the', 'transaction', 'is', 'expected', 'to', 'close', 'in', 'the', 'first', 'quarter', 'of', 'fiscal', '2020', 'this', 'is', 'in', 'line', 'with', 'infosys', 'strategy', 'to', 'strengthen', 'its', 'mortgage', 'servicing', 'capabilities', 'in', 'continental', 'europe', 'the', 'software', 'services', 'company', 'said', 'in', 'a', 'statement', 'abn', 'amro', 'will', 'continue', 'to', 'hold', 'the', 'remaining', '25', 'per', 'cent', 'stake', 'in', 'stater', 'which', 'operates', 'in', 'the', 'netherlands', 'belgium', 'and', 'germ

In [44]:
print(converted_texts[0])

[39, 1617, 4257, 5391, 999, 10, 296, 583, 18, 889, 112, 21, 11, 105, 18, 76, 907, 5, 1898, 842, 4402, 1424, 9, 6276, 6277, 228, 8838, 10, 4739, 434, 889, 958, 13, 12367, 199, 127, 5923, 12368, 2703, 127, 4257, 41, 5652, 4, 1424, 9, 9705, 8838, 186, 958, 4257, 5392, 14424, 5391, 8, 4, 5159, 14, 418, 6, 455, 9, 4, 68, 873, 7, 5393, 2591, 28, 14, 9, 471, 16, 4257, 1755, 6, 6635, 49, 4739, 12369, 6636, 9, 5394, 1618, 4, 2346, 889, 112, 21, 9, 5, 182, 6276, 6277, 41, 734, 6, 760, 4, 2471, 570, 842, 4402, 1424, 9, 9705, 60, 4740, 9, 4, 7086, 4103, 8, 1756, 4257, 51, 3326, 113, 334, 842, 4402, 26, 394, 3327, 561, 108, 4, 5653, 2756, 312, 15, 54, 1458, 3327, 842, 4402]


In [45]:
# seems to have worked well. 
print(summarizer_data_utils.convert_inds_to_text(converted_texts[0], ind2word))
print(summarizer_data_utils.convert_inds_to_text(converted_summaries[0], ind2word))

['new', 'delhi', 'infosys', 'ltd', 'india', 's', 'second', 'biggest', 'it', 'services', 'company', 'said', 'on', 'thursday', 'it', 'would', 'buy', 'a', '75', 'per', 'cent', 'stake', 'in', 'abn', 'amro', 'group', 'nv', 's', 'mortgage', 'administration', 'services', 'unit', 'for', '127', '5', 'million', 'euros', '143', '53', 'million', 'infosys', 'will', 'acquire', 'the', 'stake', 'in', 'stater', 'nv', 'through', 'unit', 'infosys', 'consulting', 'pvt', 'ltd', 'and', 'the', 'transaction', 'is', 'expected', 'to', 'close', 'in', 'the', 'first', 'quarter', 'of', 'fiscal', '2020', 'this', 'is', 'in', 'line', 'with', 'infosys', 'strategy', 'to', 'strengthen', 'its', 'mortgage', 'servicing', 'capabilities', 'in', 'continental', 'europe', 'the', 'software', 'services', 'company', 'said', 'in', 'a', 'statement', 'abn', 'amro', 'will', 'continue', 'to', 'hold', 'the', 'remaining', '25', 'per', 'cent', 'stake', 'in', 'stater', 'which', 'operates', 'in', 'the', 'netherlands', 'belgium', 'and', 'germ

## The Model
Now we can build and train our model. First we define the hyperparameters we want to use. Then we create our Summarizer and call the function .build_graph(), which as the name suggests, builds the computation graph. Then we can train the model using .train()

After training we can try our model using .infer()

In [46]:
# model hyperparameters
num_layers_encoder = 4
num_layers_decoder = 4
rnn_size_encoder = 300
rnn_size_decoder = 300

batch_size = 32
epochs = 100
clip = 5
keep_probability = 0.8
learning_rate = 0.0005
max_lr=0.005
learning_rate_decay_steps = 100
learning_rate_decay = 0.90


pretrained_embeddings_path = './tf_hub_embedding_headlinesA.npy'
summary_dir = os.path.join('./tensorboard/modelA')

use_cyclic_lr = True
inference_targets=True

In [47]:
# build graph and train the model 
summarizer_model_utils.reset_graph()
summarizer = Summarizer.Summarizer(word2ind,
                                   ind2word,
                                   save_path='./models/headlines/my_modelA',
                                   mode='TRAIN',
                                   num_layers_encoder = num_layers_encoder,
                                   num_layers_decoder = num_layers_decoder,
                                   rnn_size_encoder = rnn_size_encoder,
                                   rnn_size_decoder = rnn_size_decoder,
                                   batch_size = batch_size,
                                   clip = clip,
                                   keep_probability = keep_probability,
                                   learning_rate = learning_rate,
                                   max_lr=max_lr,
                                   learning_rate_decay_steps = learning_rate_decay_steps,
                                   learning_rate_decay = learning_rate_decay,
                                   epochs = epochs,
                                   pretrained_embeddings_path = pretrained_embeddings_path,
                                   use_cyclic_lr = use_cyclic_lr,
                                   summary_dir = summary_dir)           

summarizer.build_graph()
summarizer.train(converted_texts, 
                 converted_summaries)

Loaded pretrained embeddings.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.


W0402 15:19:29.742573 140079934850880 deprecation.py:323] From /home/mitali/Summarizer.py:162: LSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.


Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API


W0402 15:19:29.754791 140079934850880 deprecation.py:323] From /home/mitali/Summarizer.py:259: bidirectional_dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API


Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


...

Iteration: 210 of 241	train_loss: 2.1898
Iteration: 212 of 241	train_loss: 2.0653
Iteration: 214 of 241	train_loss: 1.4741
Iteration: 216 of 241	train_loss: 1.8724
Iteration: 218 of 241	train_loss: 1.5383
Iteration: 220 of 241	train_loss: 1.6384
Iteration: 222 of 241	train_loss: 1.7493
Iteration: 224 of 241	train_loss: 1.4482
Iteration: 226 of 241	train_loss: 1.6601
Iteration: 228 of 241	train_loss: 1.9351
Iteration: 230 of 241	train_loss: 1.5334
Iteration: 232 of 241	train_loss: 1.5704
Iteration: 234 of 241	train_loss: 1.7886
Iteration: 236 of 241	train_loss: 2.0578
Iteration: 238 of 241	train_loss: 1.8475
Iteration: 240 of 241	train_loss: 1.1161
Iteration: 241 of 241	train_loss: 1.4374
Average Score for this Epoch: 1.6245315074920654
-------------------- Epoch 94 of 100 --------------------
Iteration: 0 of 241	train_loss: 1.8476
Iteration: 2 of 241	train_loss: 1.4775
Iteration: 4 of 241	train_loss: 1.4434
Iteration: 6 of 241	train_loss: 1.6254
Iteration: 8 of 241	train_loss: 1.6704
I

Iteration: 116 of 241	train_loss: 1.4277
Iteration: 118 of 241	train_loss: 1.6493
Iteration: 120 of 241	train_loss: 1.5501
Iteration: 122 of 241	train_loss: 1.4860
Iteration: 124 of 241	train_loss: 1.7398
Iteration: 126 of 241	train_loss: 1.8187
Iteration: 128 of 241	train_loss: 1.3584
Iteration: 130 of 241	train_loss: 1.6270
Iteration: 132 of 241	train_loss: 1.6613
Iteration: 134 of 241	train_loss: 1.4232
Iteration: 136 of 241	train_loss: 1.9151
Iteration: 138 of 241	train_loss: 1.8390
Iteration: 140 of 241	train_loss: 1.6884
Iteration: 142 of 241	train_loss: 1.7753
Iteration: 144 of 241	train_loss: 1.3854
Iteration: 146 of 241	train_loss: 1.6949
Iteration: 148 of 241	train_loss: 2.0316
Iteration: 150 of 241	train_loss: 1.4137
Iteration: 152 of 241	train_loss: 1.2846
Iteration: 154 of 241	train_loss: 1.8753
Iteration: 156 of 241	train_loss: 1.7607
Iteration: 158 of 241	train_loss: 1.4040
Iteration: 160 of 241	train_loss: 1.5431
Iteration: 162 of 241	train_loss: 1.6293
Iteration: 164 o

Iteration: 20 of 241	train_loss: 1.2624
Iteration: 22 of 241	train_loss: 1.8075
Iteration: 24 of 241	train_loss: 1.5397
Iteration: 26 of 241	train_loss: 1.8765
Iteration: 28 of 241	train_loss: 1.7527
Iteration: 30 of 241	train_loss: 1.5802
Iteration: 32 of 241	train_loss: 1.3228
Iteration: 34 of 241	train_loss: 1.4557
Iteration: 36 of 241	train_loss: 1.4489
Iteration: 38 of 241	train_loss: 1.5360
Iteration: 40 of 241	train_loss: 1.6705
Iteration: 42 of 241	train_loss: 1.7159
Iteration: 44 of 241	train_loss: 1.6299
Iteration: 46 of 241	train_loss: 1.6775
Iteration: 48 of 241	train_loss: 1.5081
Iteration: 50 of 241	train_loss: 1.4840
Iteration: 52 of 241	train_loss: 1.7503
Iteration: 54 of 241	train_loss: 1.4476
Iteration: 56 of 241	train_loss: 1.6258
Iteration: 58 of 241	train_loss: 1.6207
Iteration: 60 of 241	train_loss: 1.4720
Iteration: 62 of 241	train_loss: 1.3625
Iteration: 64 of 241	train_loss: 1.5079
Iteration: 66 of 241	train_loss: 1.2959
Iteration: 68 of 241	train_loss: 1.1712


Iteration: 176 of 241	train_loss: 1.2227
Iteration: 178 of 241	train_loss: 1.6895
Iteration: 180 of 241	train_loss: 1.6725
Iteration: 182 of 241	train_loss: 1.6393
Iteration: 184 of 241	train_loss: 1.3322
Iteration: 186 of 241	train_loss: 1.4985
Iteration: 188 of 241	train_loss: 1.3549
Iteration: 190 of 241	train_loss: 2.0376
Iteration: 192 of 241	train_loss: 1.1883
Iteration: 194 of 241	train_loss: 1.6459
Iteration: 196 of 241	train_loss: 1.5160
Iteration: 198 of 241	train_loss: 1.3397
Iteration: 200 of 241	train_loss: 1.2993
Iteration: 202 of 241	train_loss: 1.8289
Iteration: 204 of 241	train_loss: 2.2610
Iteration: 206 of 241	train_loss: 1.5388
Iteration: 208 of 241	train_loss: 1.5555
Iteration: 210 of 241	train_loss: 1.5657
Iteration: 212 of 241	train_loss: 1.5123
Iteration: 214 of 241	train_loss: 1.3151
Iteration: 216 of 241	train_loss: 1.7473
Iteration: 218 of 241	train_loss: 1.6026
Iteration: 220 of 241	train_loss: 1.6068
Iteration: 222 of 241	train_loss: 1.7407
Iteration: 224 o

Iteration: 80 of 241	train_loss: 1.0229
Iteration: 82 of 241	train_loss: 1.4412
Iteration: 84 of 241	train_loss: 1.7134
Iteration: 86 of 241	train_loss: 1.1446
Iteration: 88 of 241	train_loss: 1.2594
Iteration: 90 of 241	train_loss: 1.3724
Iteration: 92 of 241	train_loss: 1.7085
Iteration: 94 of 241	train_loss: 1.5206
Iteration: 96 of 241	train_loss: 1.3835
Iteration: 98 of 241	train_loss: 1.3091
Iteration: 100 of 241	train_loss: 1.3919
Iteration: 102 of 241	train_loss: 1.2505
Iteration: 104 of 241	train_loss: 1.9460
Iteration: 106 of 241	train_loss: 1.7351
Iteration: 108 of 241	train_loss: 2.0143
Iteration: 110 of 241	train_loss: 1.2840
Iteration: 112 of 241	train_loss: 1.8864
Iteration: 114 of 241	train_loss: 1.7335
Iteration: 116 of 241	train_loss: 1.4207
Iteration: 118 of 241	train_loss: 1.7628
Iteration: 120 of 241	train_loss: 1.1641
Iteration: 122 of 241	train_loss: 1.5021
Iteration: 124 of 241	train_loss: 1.0817
Iteration: 126 of 241	train_loss: 1.8321
Iteration: 128 of 241	trai

## Inference

Now we can use our trained model to create summaries.

In [50]:

summarizer_model_utils.reset_graph()
summarizer = Summarizer.Summarizer(word2ind,
                                   ind2word,
                                   './models/headlines/my_modelA',
                                   'INFER',
                                   num_layers_encoder = num_layers_encoder,
                                   num_layers_decoder = num_layers_decoder,
                                   batch_size = len(converted_texts[:50]),
                                   clip = clip,
                                   keep_probability = 1.0,
                                   learning_rate = 0.0,
                                   beam_width = 5,
                                   rnn_size_encoder = rnn_size_encoder,
                                   rnn_size_decoder = rnn_size_decoder,
                                   inference_targets = False,
                                   pretrained_embeddings_path = pretrained_embeddings_path)

summarizer.build_graph()
preds = summarizer.infer(converted_texts[:50],
                         restore_path =  './models/headlines/my_modelA',
                         targets = converted_summaries[:100])

Loaded pretrained embeddings.
Graph built.
INFO:tensorflow:Restoring parameters from ./models/headlines/my_modelA


I0403 19:07:04.835104 140079934850880 saver.py:1270] Restoring parameters from ./models/headlines/my_modelA


Done.


In [51]:
# show results
summarizer_model_utils.sample_results(preds,
                                      ind2word,
                                      word2ind,
                                      converted_summaries[:50],
                                      converted_texts[:50])





 ----------------------------------------------------------------------------------------------------
Actual Text:
new delhi infosys ltd india s second biggest it services company said on thursday it would buy a 75 per cent stake in abn amro group nv s mortgage administration services unit for 127 5 million euros 143 53 million infosys will acquire the stake in stater nv through unit infosys consulting pvt ltd and the transaction is expected to close in the first quarter of fiscal 2020 this is in line with infosys strategy to strengthen its mortgage servicing capabilities in continental europe the software services company said in a statement abn amro will continue to hold the remaining 25 per cent stake in stater which operates in the netherlands belgium and germany infosys had gained 1 6 per cent by 12 58 pm while the broader mumbai market was up 0 58 per cent

Actual Summary:
abn amro infosys to buy 75 stake in abn amro s mortgage services arm times of india

Created Summary:
abn